## Plotting functions

The following functions are for the use of Adrian Liu for plotting functions that are generated with the data from Hugo's converter, especially with the parameter_2D_space function

### Generating new data

(See more info in the general presentation notebook back in the repository)

If you ever need to rerun the function to generate data, you can use the parameter_2D_space function. This was the funtions with the argument as used with the current available data (zeta : 30 to 66, Tvir: 4.0 to 4.9)


In [ ]:
#To run only if you want to regenerate the data (by changing somethings)
Heff_range = np.linspace(66, 30,10, endpoint=True)
T_vir_range = np.linspace(4.0,4.9,10, endpoint=True)
parameter_2Dspace_run('HII_EFF_FACTOR', Heff_range, 'ION_Tvir_MIN', T_vir_range, '2D_parameter_space_study_HEFF_25to52_Tvir_40to49')

#There aslo the possibility to add James algorithm. To do so, functions exist to add JAMES observables to the structures 
#(see zreion_compatison module)

## Import the data 

The following code will import the necessary data. As a reminder, the data for 2D parameter space is a 2D array filled with objects, containing the following structure. Where the array for a varying astrophysical input1 and input 2 goes, the data storage in OOP goes like this : ![title](img/hugo_converter.png)

The James structure (when added to the data) follows the eaxct same structure and has the same structure as z-reion, but with the code name jamesinfo instead of zreioninfo

In [3]:
#importing the data and necessary plotting module
import numpy as np 
import matplotlib.pyplot as plt
data_object_2D = np.load('Heff25to52_Tvir38to47_varstudy_withzreionbt_withJamesbt_withionhist_corrected.npy', allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Heff25to52_Tvir38to47_varstudy_withzreionbt_withJamesbt_withionhist_corrected.npy'

## The functions

The following functions are for generating plots like the ones provided in the original Hugo converter. If you ever need it, I will provide the original functions. Here is a list of provided plotting observable 

1. Redshift of reionization power spectrum (can me in delta² mode or not)
2. Ionization history
3. Brightness temperature field power spectrum (and it's associated movie GIF function)
4. best-fitted parameter values 
5. TAU and TAU fractional difference


#### Here are the functions : 


There is two ways of running the plotting functions :

1. Run them from the Hugo converter (Use next code block to import it)

2. Run the functions directly from this notebook (Use the code block just before all the plotting functions)



In [ ]:
import project_driver as hc

In [2]:
#The following function plots every float value (like best fitted parameters, 1 point statistics, etc. )
import imageio #Can be commented out if you don't use the GIF feature 



def analyze_float_value(obj, model, observable, Xrange, Yrange,
                        field_names=['Virial temperature [log_10(K)]', 'Ionizing effiency'], redshit_bt=20,
                        savefig=False, filenames=[]):
    '''
    This function look at the 2D variational range of a given parameter given an 2D array filled with objects
    :param obj: [arr] 2D, the object array filled with info of 21cmFAST and zreion
    :param model: [string] the name of the analyzed model (21cmFAST or zreion) (If you want both pick :"cmFAST" with add_zreion option)
    :param observable: [string] the name of the analyzed field (like z_mean or alpha parameter)
    :param Xrange: [arr] the 1D array of the Xrange
    :param Yrange: [arr] the 1D array of the Yrange
    :param field names: [list] the 2 element list of field names (default Heff and Tvir)
    :param redshit_bt: [int] the index nb of the redshfit observes if chosen 1 point statistics for the brightness temperature
    :param savefig: [bool] will save the fig and append the filename to the returned list if True. is usefull when making movies.
    :param filenames: [list] A list of filenames to append the filename of the saved figure to.
    :return: a 2D contour plot of the given field
    '''
    X, Y = np.meshgrid(Xrange, Yrange)
    obj_field = np.ones((len(Xrange), len(Yrange)))
    for i in range(len(Xrange)):
        for j in range(len(Yrange)):
            if observable == 'ion_hist':
                obj_field[i][j] = pp.compute_tau(getattr(getattr(obj[i][j], f'{model}info'), observable),
                                                 redshifts=np.linspace(5, 15,
                                                                       len(getattr(getattr(obj[i][j], f'{model}info'),
                                                                                   observable))))
            elif observable == 'brightness temperature mean':
                obj_field[i][j] = getattr(getattr(obj[i][j], f'{model}info'), 'bt_mean')[redshit_bt]
            elif observable == 'brightness temperature std':
                obj_field[i][j] = getattr(getattr(obj[i][j], f'{model}info'), 'bt_std')[redshit_bt]
            else:
                obj_field[i][j] = getattr(getattr(obj[i][j], f'{model}info'), observable)

    fig, ax = plt.subplots()
    plt.contourf(X, Y, obj_field)
    plt.colorbar()
    ax.set_xlabel(field_names[0])
    ax.set_ylabel(field_names[1])
    if observable == 'ion_hist': observable = 'TAU'
    if observable == 'brightness temperature mean' or observable == 'brightness temperature std': plt.title(
        f'{observable} variational range for {model} at a redshift of {redshit_bt}')
    plt.title(f'{observable} variational range for {model}')
    if savefig:
        manager = plt.get_current_fig_manager()
        manager.full_screen_toggle()
        plt.savefig('./bt_map/bt1James_z{}.png'.format(redshit_bt))
        filenames.append('./bt_map/bt1James_z{}.png'.format(redshit_bt))
        plt.close()
        return filenames
    else:
        plt.show()

        
        
#This next functions is used to plot 2D variational plots of powerspectrums across the parameter range. 
#It can be used for the redshfit of reionization or the brightness temperature


def plot_variational_PS(obj, model, observable, Xrange, Yrange, redshift=None, slice=None,
                        xaxis=np.logspace(np.log10(0.08570025), np.log10(7.64144032), 19), add_zreion=False,
                        add_James=False, delta2=False, field_names=['Tvir', 'Heff'], log_scale=True, savefig=False,
                        filenames=[]):
    '''
    This function plots the power spectrum over the 2D variational range of input parameters. If you want to plot the 2 mdoels togheter, use cmFASt as model with the option add zreion
    :param obj: [arr] 2D, the object array filled with info of 21cmFAST and zreion
    :param model: [string] the name of the analyzed model (21cmFAST or zreion) (If you want both pick :"cmFAST" with add_zreion option)
    :param observable: [string] the name of the field to analyze
    :param Xrange: [arr] the 1D array of the Xrange
    :param Yrange: [arr] the 1D array of the Yrange
    :param xaxis: [arr] the x axis array (default is k range for 143³ box)
    :param add_zreion: [bool] add the zreion bias if True
    :param log_scale: [bool] return log scale if True
    :param delta2: [bool] compute the delta square instead of regular powe spectrum if True (P(k)*k³ / (2*pi²))
    :param field names: [list] the 2 element list of field names (default Heff and Tvir)
    :return: a 2D contour plot of the given field
    '''

    fig, ax = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(19, 9.5))
    for i in range(len(Xrange)):
        for j in range(len(Yrange)):
            cmFastPP = getattr(getattr(obj[i][j], f'{model}info'), observable)
        
            if add_zreion:
                zreion_PP = getattr(getattr(obj[i][j], f'zreioninfo'), observable)
            if add_James:
                James_PP = getattr(getattr(obj[i][j], f'Jamesinfo'), observable)
            if observable == 'brightnesstemp':
                cmFastPP = cmFastPP[slice]
                if add_zreion: zreion_PP = zreion_PP[slice]
                if add_James: James_PP = James_PP[slice]
            if delta2:
                cmFastPP = cmFastPP * (xaxis ** 3) / (np.pi ** 2 * 2)
                if add_zreion: zreion_PP = zreion_PP * (xaxis ** 3) / (np.pi ** 2 * 2)
                if add_James: James_PP = James_PP * (xaxis ** 3) / (np.pi ** 2 * 2)
            ax[i, j].plot(xaxis, cmFastPP, label='21cmFAST')
            ax[i, j].set_xlabel(r'k [$\frac{1}{Mpc}$]')
            if add_zreion: ax[i, j].plot(xaxis, zreion_PP, label='Hugo converter')
            if add_James: ax[i, j].plot(xaxis, James_PP, label='James algorithm')

    if log_scale: plt.loglog()
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    fig.legend(lines[:3], labels[:3])
    if observable == 'brightnesstemp':
        fig.text(0.4, 0.9, f'Power spectrum of the Brightness temperature at redshift z = {redshift} ', size='large')
    else:
        fig.text(0.45, 0.9, f'Power spectrum of the redshift of reionization field', size='large')
    fig.text(0.5, 0.02, field_names[0], ha='center')
    Yrange = [round(item, 2) for item in Yrange]
    Xrange = [round(item, 2) for item in Xrange]
    fig.text(0.5, 0.032
             , f'                 {Yrange[0]}'
               f'                              {Yrange[1]}'
               f'                            {Yrange[2]}'
               f'                            {Yrange[3]}'
               f'                            {Yrange[4]}'
               f'                            {Yrange[5]}'
               f'                            {Yrange[6]}'
               f'                            {Yrange[7]}'
               f'                            {Yrange[8]}'
               f'                            {Yrange[9]}', ha='center')
    fig.text(0.04, 0.5, 'Ionizing efficiency', va='center', rotation='vertical')
    fig.text(0.06, 0.5, f'{Xrange[0]}'
                        f'             {Xrange[1]}'
                        f'             {Xrange[2]}'
                        f'             {Xrange[3]}'
                        f'             {Xrange[4]}'
                        f'             {Xrange[5]}'
                        f'             {Xrange[6]}'
                        f'             {Xrange[7]}'
                        f'             {Xrange[8]}'
                        f'             {Xrange[9]}', va='center', rotation='vertical')

    if log_scale: plt.loglog()
    if savefig:
        manager = plt.get_current_fig_manager()
        manager.full_screen_toggle()
        plt.savefig('./bt_map/bt1James_z{}.png'.format(redshift))
        filenames.append('./bt_map/bt1James_z{}.png'.format(redshift))
        plt.close()
        return filenames
    else:
        plt.show()

        
## The following function is to plot multiple ionization histories: 
    def plot_variational_ion_hist(obj, model, observable, Xrange, Yrange, xaxis='redshifts', add_zreion=False,
                              add_James=False, plot_diff=False, field_names=['Tvir', 'Heff'], log_scale=False):
    '''
    This function plots the ionization hsitories over the 2D variational range of input parameters.
    :param obj: [arr] 2D, the object array filled with info of 21cmFAST and zreion
    :param observable: [string] the name of the field to analyze
    :param Xrange: [arr] the 1D array of the Xrange
    :param Yrange: [arr] the 1D array of the Yrange
    :param xaxis: [arr] the x axis array (default is k range for 143³ box)
    :param add_zreion: [bool] add the zreion bias if True
    :param log_scale: [bool] return log scale if True
    :param plot_diff: [bool] plot the differences in the ioniozation history from the 2 models instead of the 2 individuals ioniozation histories.
    :param field names: [list] the 2 element list of field names (default Heff and Tvir)
    :return: a 2D contour plot of the given field
    '''

    fig, ax = plt.subplots(10, 10, sharex=True, sharey=True)
    for i in range(len(Xrange)):
        for j in range(len(Yrange)):
            # if xaxis == 'redshifts': xaxis = np.linspace(5, 15,len(getattr(getattr(obj[i][j], f'{model}info'), observable)))
            if xaxis == 'redshifts': xaxis = np.linspace(5, 18, 60, endpoint=True)
            if plot_diff:
                ax[i, j].plot(xaxis, np.array(getattr(getattr(obj[i][j], f'{model}info'), observable)) - np.array(
                    getattr(getattr(obj[i][j], f'zreioninfo'), observable)))
            else:
                ax[i, j].plot(xaxis, getattr(getattr(obj[i][j], f'{model}info'), observable), label=f'{model}')
                ax[i, j].set_xlabel(r'redshift')
                if j == 0:
                    ax[i, j].set_ylabel(r'$x_i$', size='small')
                if add_zreion:
                    ax[i, j].plot(xaxis, getattr(getattr(obj[i][j], f'zreioninfo'), observable), label='Hugo algorithm')
                if add_James:
                    ax[i, j].plot(xaxis, getattr(getattr(obj[i][j], f'Jamesinfo'), observable), label='James algorithm')

    # ax.set_xlabel(field_names[0])
    # ax.set_ylabel(field_names[1])
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    fig.legend(lines[:3], labels[:3])
    fig.text(0.45, 0.9, f'Ionization history as function of the ioniozation efficiency and virial temperature',
             size='large')
    if log_scale: plt.loglog()
    fig.text(0.5, 0.01, r'Virial temperature [$log_{10}(K)$]', ha='center')

    Yrange = [round(item, 2) for item in Yrange]
    Xrange = [round(item, 2) for item in Xrange]
    fig.text(0.5, 0.032
             , f'                 {Yrange[0]}'
               f'                              {Yrange[1]}'
               f'                            {Yrange[2]}'
               f'                            {Yrange[3]}'
               f'                            {Yrange[4]}'
               f'                            {Yrange[5]}'
               f'                            {Yrange[6]}'
               f'                            {Yrange[7]}'
               f'                            {Yrange[8]}'
               f'                            {Yrange[9]}', ha='center')
    fig.text(0.04, 0.5, 'Ionizing efficiency', va='center', rotation='vertical')
    fig.text(0.06, 0.5, f'{Xrange[0]}'
                        f'             {Xrange[1]}'
                        f'             {Xrange[2]}'
                        f'             {Xrange[3]}'
                        f'             {Xrange[4]}'
                        f'             {Xrange[5]}'
                        f'             {Xrange[6]}'
                        f'             {Xrange[7]}'
                        f'             {Xrange[8]}'
                        f'             {Xrange[9]}', va='center', rotation='vertical')


    plt.show()
    

# The following function makes a movie with the brightness temperature

def make_bt_movie(stoo, k_values, Heff_range, T_vir_range, gifname, add_zreion=True, add_James=True):
    '''
    This function makes a brightness temperature movie
    :param stoo: the object of 2D parameter space containing all the brightness temperature information
    :param kvalues: the values of k of the power_spectrum computations
    :param add_zreion: (default True) add z-reion info if True
    :param add_James: (default True) add James' algorithm info if True
    :param gifname: [string] the name you want to give to your gif
    :return:
    '''
    filenames = []
    for slice in tqdm(range(len(stoo[0][0].cmFASTinfo.z_for_bt)), 'making a reionization movie'):
        filenames = zrcomp.plot_variational_bright_temp(stoo, 'cmFAST', 'brightnesstemp',
                                                        stoo[0][0].cmFASTinfo.z_for_bt[slice], slice, T_vir_range,
                                                        Heff_range, xaxis=k_values, add_zreion=add_zreion, savefig=True,
                                                        filenames=filenames, add_James=add_James,
                                                        )

    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(f'{gifname}.gif', images)
    return

#The following function in use for TAU computation (necessary for making the TAU analysises plots)

## generating new plots


### 1. Redshift of reionization field 

The function plot_variational_PS can be used to plot the brightness temperature, or the redshfit of reionization field. It has several options, like in including the redshift of reioniz

In [ ]:
def plot_variational_PS(obj, model, observable, Xrange, Yrange, redshift=None, slice=None,
                        xaxis=np.logspace(np.log10(0.08570025), np.log10(7.64144032), 19), add_zreion=False,
                        add_James=False, delta2=False, field_names=[r'Virial temperature [$log_{10}(K)$]', 'Ionizing efficiency'], log_scale=True, savefig=False,
                        filenames=[]):
    '''
    This function plots the power spectrum over the 2D variational range of input parameters. If you want to plot the 2 mdoels togheter, use cmFASt as model with the option add zreion
    :param obj: [arr] 2D, the object array filled with info of 21cmFAST and zreion
    :param observable: [string] the name of the field to analyze
    :param Xrange: [arr] the 1D array of the Xrange
    :param Yrange: [arr] the 1D array of the Yrange
    :param xaxis: [arr] the x axis array (default is k range for 143³ box)
    :param add_zreion: [bool] add the zreion bias if True
    :param log_scale: [bool] return log scale if True
    :param delta2: [bool] compute the delta square instead of regular powe spectrum if True (P(k)*k³ / (2*pi²))
    :param field names: [list] the 2 element list of field names (default Heff and Tvir)
    :return: a 2D contour plot of the given field
    '''

    fig, ax = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(19, 9.5))
    for i in range(len(Xrange)):
        for j in range(len(Yrange)):
            cmFastPP = getattr(getattr(obj[i][j], f'{model}info'), observable)
            # cmFastPP /= 143**3 #temporary normalization constant (V³)
            if add_zreion:
                zreion_PP = getattr(getattr(obj[i][j], f'zreioninfo'), observable)
            if add_James:
                James_PP = getattr(getattr(obj[i][j], f'Jamesinfo'), observable)
            if observable == 'brightnesstemp':
                cmFastPP = cmFastPP[slice]
                if add_zreion: zreion_PP = zreion_PP[slice]
                if add_James: James_PP = James_PP[slice]
            if delta2:
                cmFastPP = cmFastPP * (xaxis ** 3) / (np.pi ** 2 * 2)
                if add_zreion: zreion_PP = zreion_PP * (xaxis ** 3) / (np.pi ** 2 * 2)
                if add_James: James_PP = James_PP * (xaxis ** 3) / (np.pi ** 2 * 2)
            ax[i, j].plot(xaxis, cmFastPP, label='21cmFAST')
            ax[i, j].set_xlabel(r'k [$\frac{1}{Mpc}$]')
            if add_zreion: ax[i, j].plot(xaxis, zreion_PP, label='Hugo converter')
            if add_James: ax[i, j].plot(xaxis, James_PP, label='James algorithm')
    # ax.set_xlabel(field_names[0])
    # ax.set_ylabel(field_names[1])
    if log_scale: plt.loglog()
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    fig.legend(lines[:3], labels[:3])
    if observable == 'brightnesstemp':
        fig.text(0.4, 0.9, f'Power spectrum of the Brightness temperature at redshift z = {redshift} ', size='large')
    else:
        fig.text(0.45, 0.9, f'Power spectrum of the redshift of reionization field', size='large')
    # plt.title('The Power spectrum of the redshfit of reioniozation fields as a function of heff and Tvir')
    fig.text(0.5, 0.02, field_names[0], ha='center')
    Yrange = [int(item) for item in Yrange]
    Xrange = [round(item, 2) for item in Xrange]
    fig.text(0.5, 0.032
             , f'                 {Xrange[0]}'
               f'                              {Xrange[1]}'
               f'                            {Xrange[2]}'
               f'                            {Xrange[3]}'
               f'                            {Xrange[4]}'
               f'                            {Xrange[5]}'
               f'                            {Xrange[6]}'
               f'                            {Xrange[7]}'
               f'                            {Xrange[8]}'
               f'                            {Xrange[9]}', ha='center')
    fig.text(0.04, 0.5, 'Ionizing efficiency', va='center', rotation='vertical')
    fig.text(0.06, 0.5, f'{Yrange[9]}'
                        f'             {Yrange[8]}'
                        f'             {Yrange[7]}'
                        f'             {Yrange[6]}'
                        f'             {Yrange[5]}'
                        f'             {Yrange[4]}'
                        f'             {Yrange[3]}'
                        f'             {Yrange[2]}'
                        f'             {Yrange[1]}'
                        f'             {Yrange[0]}', va='center', rotation='vertical')

    if log_scale: plt.loglog()
    if savefig:
        manager = plt.get_current_fig_manager()
        manager.full_screen_toggle()
        plt.savefig('./bt_map/bt1James_z{}.png'.format(redshift))
        filenames.append('./bt_map/bt1James_z{}.png'.format(redshift))
        plt.close()
        return filenames
    else:
        plt.show()


In [1]:
#plot the power spectrums (of the redshfit of reionization field)
plot_variational_PS(data_object_2D,'cmFAST','P_k_zre', T_vir_range, Heff_range, add_zreion=True, add_James = True)
#or hc.zrcomp.plot_variational_PS(stoo,'cmFAST','P_k_zre', T_vir_range, Heff_range, add_zreion=True, add_James = True)


#you can also plot it's delta2 version

### 2. Ionization history

The function plot_variational_ion_hist can be used to plot the ionization histories of the three different models. 

In [ ]:
def plot_variational_ion_hist(obj, model, observable, Xrange, Yrange, xaxis='redshifts', add_zreion=False,
                              add_James=False, plot_diff=False, field_names=[r'Virial temperature [$log_{10}(K)$]', 'Ionizing efficiency'], log_scale=False):
    '''
    This function plots the ionization histories over the 2D variational range of input parameters.
    :param obj: [arr] 2D, the object array filled with info of 21cmFAST and zreion
    :param observable: [string] the name of the field to analyze
    :param Xrange: [arr] the 1D array of the Xrange
    :param Yrange: [arr] the 1D array of the Yrange
    :param xaxis: [arr] the x axis array (default is k range for 143³ box)
    :param add_zreion: [bool] add the zreion histories if True
    :param add_James: [bool] add the james histories if True
    :param log_scale: [bool] return log scale if True
    :param plot_diff: [bool] plot the differences in the ioniozation history from the 2 models instead of the 2 individuals ioniozation histories.
    :param field names: [list] the 2 element list of field names (default Heff and Tvir)
    :return: a 2D 10x10 grid plot of the ionization histories
    '''

    fig, ax = plt.subplots(10, 10, sharex=True, sharey=True)
    for i in range(len(Xrange)):
        for j in range(len(Yrange)):
            # if xaxis == 'redshifts': xaxis = np.linspace(5, 15,len(getattr(getattr(obj[i][j], f'{model}info'), observable)))
            if xaxis == 'redshifts': xaxis = np.linspace(5, 18, 60, endpoint=True)
            if plot_diff:
                ax[i, j].plot(xaxis, np.array(getattr(getattr(obj[i][j], f'{model}info'), observable)) - np.array(
                    getattr(getattr(obj[i][j], f'zreioninfo'), observable)))
            else:
                ax[i, j].plot(xaxis, getattr(getattr(obj[i][j], f'{model}info'), observable), label=f'{model}')
                ax[i, j].set_xlabel(r'redshift')
                if j == 0:
                    ax[i, j].set_ylabel(r'$x_i$', size='small')
                if add_zreion:
                    ax[i, j].plot(xaxis, getattr(getattr(obj[i][j], f'zreioninfo'), observable), label='Hugo algorithm')
                if add_James:
                    ax[i, j].plot(xaxis, getattr(getattr(obj[i][j], f'Jamesinfo'), observable), label='James algorithm')

    # ax.set_xlabel(field_names[0])
    # ax.set_ylabel(field_names[1])
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    fig.legend(lines[:3], labels[:3])
    fig.text(0.45, 0.9, f'Ionization history as function of the ioniozation efficiency and virial temperature',
             size='large')
    if log_scale: plt.loglog()
    fig.text(0.5, 0.01, r'Virial temperature [$log_{10}(K)$]', ha='center')

    Yrange = [int(item) for item in Yrange]
    Xrange = [round(item, 2) for item in Xrange]
    fig.text(0.5, 0.032
             , f'                 {Xrange[0]}'
               f'                              {Xrange[1]}'
               f'                            {Xrange[2]}'
               f'                            {Xrange[3]}'
               f'                            {Xrange[4]}'
               f'                            {Xrange[5]}'
               f'                            {Xrange[6]}'
               f'                            {Xrange[7]}'
               f'                            {Xrange[8]}'
               f'                            {Xrange[9]}', ha='center')
    fig.text(0.04, 0.5, 'Ionizing efficiency', va='center', rotation='vertical')
    fig.text(0.06, 0.5, f'{Yrange[9]}'
                        f'             {Yrange[8]}'
                        f'             {Yrange[7]}'
                        f'             {Yrange[6]}'
                        f'             {Yrange[5]}'
                        f'             {Yrange[4]}'
                        f'             {Yrange[3]}'
                        f'             {Yrange[2]}'
                        f'             {Yrange[1]}'
                        f'             {Yrange[0]}', va='center', rotation='vertical')
    plt.show()


In [ ]:
plot_variational_ion_hist(data_object_2D, 'cmFAST', 'ion_hist', T_vir_range, Heff_range, add_zreion = True, add_James=True, xaxis=np.linspace(5,18,60))

#### 3.  brightness temperature power spectrum 

To generate the power spectrum of a the brightness temeprature for the 2D parameter space of the three models. The function below can also be run to make a brightness temeprature spectrum as a function of redshfit. 



In [1]:
#The function for the movie
import imageio
def make_bt_movie(stoo, k_values, Heff_range, T_vir_range, gifname, add_zreion=True, add_James=True):
    '''
    This function makes a brightness temperature movie
    :param stoo: the object of 2D parameter space containing all the brightness temperature information
    :param kvalues: the values of k of the power_spectrum computations
    :param add_zreion: (default True) add z-reion info if True
    :param add_James: (default True) add James' algorithm info if True
    :param gifname: [string] the name you want to give to your gif
    :return:
    '''
    filenames = []
    for slice in tqdm(range(len(stoo[0][0].cmFASTinfo.z_for_bt)), 'making a reionization movie'):
        filenames = zrcomp.plot_variational_bright_temp(stoo, 'cmFAST', 'brightnesstemp',
                                                        stoo[0][0].cmFASTinfo.z_for_bt[slice], slice, T_vir_range,
                                                        Heff_range, xaxis=k_values, add_zreion=add_zreion, savefig=True,
                                                        filenames=filenames, add_James=add_James,
                                                        )

    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(f'{gifname}.gif', images)
    return


In [2]:
#where the slice is the bt power spectrum index corresping to the redshift you want to plot it add (goes from 1 to 60)
slice = 20
plot_variational_bright_temp(data_object_2D, 'cmFAST', 'brightnesstemp',
                                                        data_object_2D[0][0].cmFASTinfo.z_for_bt[slice], slice, T_vir_range,
                                                        Heff_range, xaxis=k_values, add_zreion=True, add_James=True,
                                                        )

#### 4. Float observables (like best fitted alpha, b_0, k_0) and mean of brightness temperatue

All the observable that are float values can be analyzed with this function, such as the best fit value parameter for alpha, b_0 and k_0

In [ ]:
def analyze_float_value(obj, model, observable, Xrange, Yrange,
                        field_names=[r'Virial temperature [$log_{10}(K)$]', 'Ionizing efficiency'], redshit_bt=15,
                        savefig=False, filenames=[]):
    '''
    This function look at the 2D variational range of a given parameter given an 2D array filled with objects
    :param obj: [arr] 2D, the object array filled with info of 21cmFAST and zreion
    :param model: [string] the name of the analyzed model (21cmFAST or zreion)
    :param observable: [string] the name of the analyzed field (like z_mean or alpha parameter)
    :param Xrange: [arr] the 1D array of the Xrange
    :param Yrange: [arr] the 1D array of the Yrange
    :param field names: [list] the 2 element list of field names (default Heff and Tvir)
    :param redshit_bt: [int] the index nb of the redshfit observes if chosen 1 point statistics for the brightness temperature
    :param savefig: [bool] will save the fig and append the filename to the returned list if True. is usefull when making movies.
    :param filenames: [list] A list of filenames to append the filename of the saved figure to.
    :return: a 2D contour plot of the given field
    '''
    X, Y = np.meshgrid(Xrange, Yrange)
    obj_field = np.ones((len(Xrange), len(Yrange)))
    for i in range(len(Xrange)):
        for j in range(len(Yrange)):
            if observable == 'ion_hist':
                obj_field[i][j] = pp.compute_tau(getattr(getattr(obj[i][j], f'{model}info'), observable),
                                                 redshifts=np.linspace(5, 15,
                                                                       len(getattr(getattr(obj[i][j], f'{model}info'),
                                                                                   observable))))
            elif observable == 'brightness temperature mean':
                obj_field[i][j] = getattr(getattr(obj[i][j], f'{model}info'), 'bt_mean')[redshit_bt]
            elif observable == 'brightness temperature std':
                obj_field[i][j] = getattr(getattr(obj[i][j], f'{model}info'), 'bt_std')[redshit_bt]
            else:
                obj_field[i][j] = getattr(getattr(obj[i][j], f'{model}info'), observable)

    fig, ax = plt.subplots()
    plt.contourf(X, Y, obj_field)
    plt.colorbar()
    ax.set_xlabel(field_names[0])
    ax.set_ylabel(field_names[1])
    if observable == 'ion_hist': observable = 'TAU'
    if observable == 'brightness temperature mean' or observable == 'brightness temperature std': plt.title(
        f'{observable} variational range for {model} at a redshift of {redshit_bt}')
    plt.title(f'{observable} variational range for {model}')
    if savefig:
        manager = plt.get_current_fig_manager()
        manager.full_screen_toggle()
        plt.savefig('./bt_map/bt1James_z{}.png'.format(redshit_bt))
        filenames.append('./bt_map/bt1James_z{}.png'.format(redshit_bt))
        plt.close()
        return filenames
    else:
        plt.show()
        return obj_field


In [ ]:
#plot the mean redshift of reionizaion from 21cmFASt and alpha variational range as an exemple
analyze_float_value(data_object_2D, 'cmFAST' , 'z_mean', T_vir_range, Heff_range)
analyze_float_value(data_object_2D, 'zreion' , 'alpha', T_vir_range, Heff_range)

#### 5. fractional differences

You can plot TAU fractional differences or simply brightness temperature mean fractional differences with these functions


In [ ]:
def analyze_frac_diff(obj, model1, model2, observable, Xrange, Yrange, field_names=[r'Virial temperature [$log_{10}(K)$]', 'Ionizing efficiency'], redshift_bt = None):
    '''
    This function lookat the fractional difference between the observable of 2  models. It works for the TAU parameter and the brightness temeprature means or stds. (Model1- Model2 / Model1) You can make it work for other params if you take out the [redshfit_bt] from the else function
    :param obj: [arr] 2D, the object array filled with info of 21cmFAST and zreion
    :param model1: [string] the name of the analyzed model (21cmFAST, James or zreion)
    :param model2: [string] the name of the analyzed model (21cmFAST, James or zreion)
    :param observable: [string] the name of the analyzed field (like z_mean or alpha parameter)
    :param Xrange: [arr] the 1D array of the Xrange
    :param Yrange: [arr] the 1D array of the Yrange
    :param field names: [list] the 2 element list of field names (default Heff and Tvir)
    :param redshift_bt: [int] the slice of brightness temperature corresponding to the desired redshift
    :return: a 2D contour plot of the fractional difference for the given field
    '''
    X, Y = np.meshgrid(Xrange, Yrange)
    obj_field = np.ones((len(Xrange), len(Yrange)))
    for i in range(len(Xrange)):
        for j in range(len(Yrange)):
            if observable == 'ion_hist':
                cmFAST_TAU = pp.compute_tau(getattr(getattr(obj[i][j], f'{model1}info'), observable),
                                            redshifts=np.linspace(5, 18,
                                                                  len(getattr(getattr(obj[i][j], f'{model1}info'),
                                                                              observable))))
                zreion_TAU = pp.compute_tau(getattr(getattr(obj[i][j], f'{model2}info'), observable),
                                            redshifts=np.linspace(5, 18,
                                                                  len(getattr(getattr(obj[i][j], f'{model1}info'),
                                                                              observable))))
                diff_TAU = cmFAST_TAU - zreion_TAU

                obj_field[i][j] = diff_TAU / cmFAST_TAU
                if obj_field[i][j] > 0.08: obj_field[i][j] = 0.08
            else:
                cmFAST_TAU = getattr(getattr(obj[i][j], f'{model1}info'), observable)[redshift_bt]
                zreion_TAU = getattr(getattr(obj[i][j], f'{model2}info'), observable)[redshift_bt]
                diff_TAU = cmFAST_TAU - zreion_TAU
                print(diff_TAU)
                obj_field[i][j] = diff_TAU / cmFAST_TAU
                #if obj_field[i][j] < -0.4 : obj_field[i][j] = -0.4


    fig, ax = plt.subplots()
    if observable == 'ion_hist':
        levels = np.linspace(obj_field.min(), 0.08, 3000)
        cntr = plt.contourf(X, Y, obj_field, levels=levels, vmin=-0.06, vmax=0.06, cmap='RdBu')
    else:
        levels = np.linspace(obj_field.min(), obj_field.max(), 3000)
        absolute_big = max(obj_field.max(), abs(obj_field.min()))
        if absolute_big > 1: absolute_big = 1
        cntr = plt.contourf(X, Y, obj_field,  levels = levels, vmin=-absolute_big, vmax=absolute_big, cmap='RdBu')
    #plt.clim(-0.06, 0.06)
    plt.colorbar(cntr, ax=ax)
    ax.set_xlabel(field_names[0])
    ax.set_ylabel(field_names[1])
    if model1 == 'zreion': model1 = 'Hugo converter'
    if model2 == 'zreion': model2 = 'Hugo converter'
    if observable == 'ion_hist': plt.title(f'TAU variational range for ({model1} - {model2}) / {model1}')
    else: plt.title(f'{observable} variational range for ({model1} - {model2}) / {model1} at a redshfit z = {obj[0][0].cmFASTinfo.z_for_bt[redshift_bt]}' )
    plt.show()


In [ ]:
analyze_frac_diff(data_object_2D, 'cmFAST','zreion', 'ion_hist',T_vir_range, Heff_range, redshift_bt=slice)
analyze_frac_diff(data_object_2D, 'cmFAST','zreion', 'ion_hist',T_vir_range, Heff_range, redshift_bt=slice)